In [9]:
import os
import re
from glob import glob
from tqdm import tqdm
import javalang
import pandas as pd


def get_source_code(commitId, project):
    import random
    import requests
    from robobrowser import RoboBrowser

    HEADERS_LIST = [
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
        'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
        'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
        'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
    ]

    link = []

    session = requests.Session()
    browser = RoboBrowser(session=session, user_agent=random.choice(HEADERS_LIST), parser="lxml")
    url = "https://github.com/" + project.replace("-", "/") + "/commit/" + commitId

    browser.open(url + "?diff=unified")
    results = browser.find_all("a")
    for item in results:
        if ".java" in str(item):
            second_url = "https://raw.githubusercontent.com/" + project.replace("-",
                                                                                "/") + "/" + commitId + "/" + item.string
            browser.open(second_url)
            return browser.find().text


def removeComments(string):
    string = re.sub(re.compile("/\*.*?\*/", re.DOTALL), "",
                    string)  # remove all occurance streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n"), "",
                    string)  # remove all occurance singleline comments (//COMMENT\n ) from string
    return string


temp_subfiles = []

pattern = "*.java"

for dir, _, _ in os.walk("/home/manny/PycharmProjects/GithubScraper/java_code_files"):
    temp_subfiles.extend(glob(os.path.join(dir, pattern)))

# print(temp_subfiles)

if (False):
    for file in tqdm(temp_subfiles):
        with open(file, 'r') as myfile:
            data = myfile.read()  # .replace('\n', '')
            # data = removeComments(data)
            tree = javalang.parse.parse(data)
            # for codeblock in tree.children:
            #     print(codeblock)
            #     print("===============")
            #     for proto in codeblock:
            #         print(proto)
            #
            #         print("*************")
            #
            # tokens = list(javalang.tokenizer.tokenize(data))
            # for token in tokens:
            #     # print(token.value)
            #     # print(token.position)
            #     print(type(token))
import os
import re
from glob import glob
from tqdm import tqdm
import javalang
import pandas as pd


def get_source_code(commitId, project):
    import random
    import requests
    from robobrowser import RoboBrowser

    HEADERS_LIST = [
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
        'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
        'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
        'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
    ]

    link = []
    codes = []

    while (len(codes) < 1):
        session = requests.Session()
        browser = RoboBrowser(session=session, user_agent=random.choice(HEADERS_LIST), parser="lxml")
        url = "https://github.com/" + project.replace("-", "/") + "/commit/" + commitId

        browser.open(url + "?diff=unified")

        results = browser.find_all("a")
        for item in results:
            if ".java" in str(item):
                try:
                    #             print(item.string)
                    second_url = "https://raw.githubusercontent.com/" + project.replace("-",
                                                                                        "/") + "/" + commitId + "/" + item.string
                    browser.open(second_url)
                    codes.append(browser.find().text)
                except:
                    pass

    return codes



def removeComments(string):
    string = re.sub(re.compile("/\*.*?\*/", re.DOTALL), "",
                    string)  # remove all occurance streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n"), "",
                    string)  # remove all occurance singleline comments (//COMMENT\n ) from string
    return string


code = {"JavaSyntaxError": [], "code_col": [], "code_line": [], "code": [], "astcode": [], 'database_source': []}


temp_subfiles = []

pattern = "*.java"

for dir, _, _ in os.walk("/home/manny/PycharmProjects/GithubScraper/java_code_files"):
    temp_subfiles.extend(glob(os.path.join(dir, pattern)))

# print(temp_subfiles)

if (True):
    for file in tqdm(temp_subfiles):
        with open(file, 'r') as myfile:
            data = myfile.read()  # .replace('\n', '')
            # data = removeComments(data)
            try:
                if data!= None:
                    try:
                        tree = javalang.parse.parse(data)
                    except:
                        pass
                    for codeblock in tree.children:
                        try:
                            code['code'].append(codeblock._position._source)
                        except AttributeError:
                            code['code'].append(None)
                        try:
                            code['code_line'].append(codeblock._position.line)
                        except AttributeError:
                            code['code_line'].append(None)
                        try:
                            code['code_col'].append(codeblock._position.column)
                        except AttributeError:
                            code['code_col'].append(None)

                        code['astcode'].append(codeblock)
                        code['database_source'].append("githubrepos")
                        code['JavaSyntaxError'].append(False)

            except (javalang.parser.JavaSyntaxError, KeyError, TypeError):
                code['code'].append(data)
                code['astcode'].append(None)
                code['code_line'].append(None)
                code['code_col'].append(None)
                code['JavaSyntaxError'].append(True)
                code['database_source'].append("Promise")
                pass
            # for codeblock in tree.children:
            #     print(codeblock)
            #     print("===============")
            #     for proto in codeblock:
            #         print(proto)
            #
            #         print("*************")
            #
            # tokens = list(javalang.tokenizer.tokenize(data))
            # for token in tokens:
            #     # print(token.value)
            #     # print(token.position)
            #     print(type(token))


def dict_append(code, key, value):
    if value == None:
        code[key].append("nan")
    else:
        code[key].append("value")
        return








  0%|          | 0/52598 [00:00<?, ?it/s]





  0%|          | 5/52598 [00:00<19:07, 45.83it/s]





  0%|          | 21/52598 [00:00<08:54, 98.28it/s]





  0%|          | 36/52598 [00:00<07:40, 114.16it/s]





  0%|          | 51/52598 [00:00<07:10, 122.16it/s]





  0%|          | 62/52598 [00:00<07:38, 114.47it/s]





  0%|          | 86/52598 [00:00<06:42, 130.32it/s]





  0%|          | 99/52598 [00:00<08:08, 107.49it/s]





  0%|          | 113/52598 [00:01<07:57, 109.99it/s]





  0%|          | 125/52598 [00:01<08:25, 103.83it/s]





  0%|          | 135/52598 [00:01<08:33, 102.10it/s]





  0%|          | 145/52598 [00:01<08:43, 100.19it/s]





  0%|          | 156/52598 [00:01<08:41, 100.59it/s]





  0%|          | 172/52598 [00:01<08:24, 104.01it/s]





  0%|          | 184/52598 [00:01<08:24, 103.80it/s]





  0%|          | 195/52598 [00:02<09:06, 95.92it/s] 





  0%|          | 207/52598 [00:02<08:59, 97.03it/s]





  0%|          | 223/52598 [00

  3%|▎         | 1379/52598 [00:19<11:50, 72.08it/s]





  3%|▎         | 1391/52598 [00:19<11:48, 72.30it/s]





  3%|▎         | 1403/52598 [00:19<11:47, 72.34it/s]





  3%|▎         | 1414/52598 [00:20<12:13, 69.74it/s]





  3%|▎         | 1432/52598 [00:20<12:08, 70.28it/s]





  3%|▎         | 1446/52598 [00:20<12:04, 70.56it/s]





  3%|▎         | 1462/52598 [00:20<12:00, 70.93it/s]





  3%|▎         | 1474/52598 [00:20<12:02, 70.81it/s]





  3%|▎         | 1487/52598 [00:20<11:58, 71.09it/s]





  3%|▎         | 1498/52598 [00:21<11:57, 71.24it/s]





  3%|▎         | 1509/52598 [00:21<11:56, 71.27it/s]





  3%|▎         | 1520/52598 [00:21<11:54, 71.46it/s]





  3%|▎         | 1532/52598 [00:21<11:52, 71.68it/s]





  3%|▎         | 1546/52598 [00:21<11:49, 71.96it/s]





  3%|▎         | 1561/52598 [00:21<11:45, 72.31it/s]





  3%|▎         | 1574/52598 [00:21<11:44, 72.48it/s]





  3%|▎         | 1588/52598 [00:21<11:41, 72.71it/s]





  3%|▎        

  5%|▌         | 2839/52598 [00:41<12:06, 68.50it/s]





  5%|▌         | 2853/52598 [00:41<12:04, 68.67it/s]





  5%|▌         | 2872/52598 [00:41<12:01, 68.94it/s]





  5%|▌         | 2886/52598 [00:41<12:01, 68.94it/s]





  6%|▌         | 2898/52598 [00:42<12:03, 68.71it/s]





  6%|▌         | 2908/52598 [00:42<12:02, 68.76it/s]





  6%|▌         | 2923/52598 [00:42<12:00, 68.95it/s]





  6%|▌         | 2934/52598 [00:42<11:59, 68.99it/s]





  6%|▌         | 2944/52598 [00:42<11:59, 69.01it/s]





  6%|▌         | 2957/52598 [00:42<11:57, 69.15it/s]





  6%|▌         | 2971/52598 [00:42<11:56, 69.29it/s]





  6%|▌         | 2983/52598 [00:42<11:54, 69.40it/s]





  6%|▌         | 2997/52598 [00:43<11:54, 69.47it/s]





  6%|▌         | 3009/52598 [00:43<11:52, 69.55it/s]





  6%|▌         | 3024/52598 [00:43<11:51, 69.64it/s]





  6%|▌         | 3045/52598 [00:44<12:11, 67.71it/s]





  6%|▌         | 3053/52598 [00:45<12:11, 67.73it/s]





  6%|▌        

  8%|▊         | 4410/52598 [01:02<11:19, 70.93it/s]





  8%|▊         | 4419/52598 [01:02<11:19, 70.89it/s]





  8%|▊         | 4430/52598 [01:02<11:18, 70.95it/s]





  8%|▊         | 4448/52598 [01:02<11:17, 71.10it/s]





  8%|▊         | 4459/52598 [01:02<11:17, 71.02it/s]





  9%|▊         | 4474/52598 [01:02<11:16, 71.14it/s]





  9%|▊         | 4492/52598 [01:03<11:15, 71.25it/s]





  9%|▊         | 4513/52598 [01:03<11:12, 71.47it/s]





  9%|▊         | 4527/52598 [01:03<11:12, 71.53it/s]





  9%|▊         | 4545/52598 [01:03<11:10, 71.70it/s]





  9%|▊         | 4565/52598 [01:03<11:08, 71.90it/s]





  9%|▊         | 4581/52598 [01:03<11:06, 72.00it/s]





  9%|▊         | 4597/52598 [01:03<11:05, 72.11it/s]





  9%|▉         | 4612/52598 [01:03<11:05, 72.14it/s]





  9%|▉         | 4629/52598 [01:04<11:03, 72.30it/s]





  9%|▉         | 4647/52598 [01:04<11:01, 72.46it/s]





  9%|▉         | 4662/52598 [01:04<11:00, 72.57it/s]





  9%|▉        

 11%|█         | 5781/52598 [01:21<10:56, 71.35it/s]





 11%|█         | 5792/52598 [01:21<10:55, 71.36it/s]





 11%|█         | 5803/52598 [01:21<10:55, 71.38it/s]





 11%|█         | 5811/52598 [01:21<10:55, 71.37it/s]





 11%|█         | 5822/52598 [01:21<10:55, 71.40it/s]





 11%|█         | 5834/52598 [01:21<10:54, 71.46it/s]





 11%|█         | 5844/52598 [01:21<10:54, 71.47it/s]





 11%|█         | 5853/52598 [01:21<10:54, 71.46it/s]





 11%|█         | 5862/52598 [01:22<10:54, 71.41it/s]





 11%|█         | 5871/52598 [01:22<10:54, 71.43it/s]





 11%|█         | 5879/52598 [01:22<10:54, 71.33it/s]





 11%|█         | 5888/52598 [01:22<10:54, 71.35it/s]





 11%|█         | 5897/52598 [01:22<10:54, 71.36it/s]





 11%|█         | 5912/52598 [01:22<10:53, 71.45it/s]





 11%|█▏        | 5928/52598 [01:22<10:52, 71.54it/s]





 11%|█▏        | 5945/52598 [01:25<11:12, 69.42it/s]





 11%|█▏        | 5953/52598 [01:25<11:11, 69.43it/s]





 11%|█▏       

...

 89%|████████▊ | 46656/52598 [14:44<01:52, 52.74it/s]





 89%|████████▊ | 46667/52598 [14:44<01:52, 52.75it/s]





 89%|████████▊ | 46678/52598 [14:44<01:52, 52.76it/s]





 89%|████████▉ | 46689/52598 [14:44<01:51, 52.76it/s]





 89%|████████▉ | 46699/52598 [14:45<01:51, 52.77it/s]





 89%|████████▉ | 46710/52598 [14:45<01:51, 52.77it/s]





 89%|████████▉ | 46720/52598 [14:45<01:51, 52.77it/s]





 89%|████████▉ | 46729/52598 [14:45<01:51, 52.77it/s]





 89%|████████▉ | 46740/52598 [14:45<01:50, 52.78it/s]





 89%|████████▉ | 46749/52598 [14:45<01:50, 52.78it/s]





 89%|████████▉ | 46765/52598 [14:45<01:50, 52.79it/s]





 89%|████████▉ | 46777/52598 [14:45<01:50, 52.80it/s]





 89%|████████▉ | 46788/52598 [14:46<01:50, 52.80it/s]





 89%|████████▉ | 46798/52598 [14:46<01:49, 52.80it/s]





 89%|████████▉ | 46807/52598 [14:46<01:49, 52.81it/s]





 89%|████████▉ | 46816/52598 [14:46<01:49, 52.81it/s]





 89%|████████▉ | 46831/52598 [14:46<01:49, 52.82it/s]




 91%|█████████ | 47809/52598 [14:58<01:29, 53.22it/s]





 91%|█████████ | 47821/52598 [14:58<01:29, 53.23it/s]





 91%|█████████ | 47834/52598 [14:58<01:29, 53.23it/s]





 91%|█████████ | 47846/52598 [14:58<01:29, 53.24it/s]





 91%|█████████ | 47857/52598 [14:58<01:29, 53.25it/s]





 91%|█████████ | 47868/52598 [14:58<01:28, 53.25it/s]





 91%|█████████ | 47880/52598 [14:59<01:28, 53.26it/s]





 91%|█████████ | 47891/52598 [14:59<01:28, 53.27it/s]





 91%|█████████ | 47902/52598 [14:59<01:28, 53.27it/s]





 91%|█████████ | 47915/52598 [14:59<01:27, 53.27it/s]





 91%|█████████ | 47928/52598 [14:59<01:27, 53.28it/s]





 91%|█████████ | 47941/52598 [14:59<01:27, 53.29it/s]





 91%|█████████ | 47957/52598 [14:59<01:27, 53.30it/s]





 91%|█████████ | 47970/52598 [14:59<01:26, 53.31it/s]





 91%|█████████ | 47986/52598 [15:00<01:26, 53.32it/s]





 91%|█████████▏| 47998/52598 [15:00<01:26, 53.32it/s]





 91%|█████████▏| 48010/52598 [15:00<01:26, 53.33it/s]




 94%|█████████▍| 49594/52598 [15:11<00:55, 54.39it/s]





 94%|█████████▍| 49615/52598 [15:12<00:54, 54.40it/s]





 94%|█████████▍| 49639/52598 [15:12<00:54, 54.42it/s]





 94%|█████████▍| 49658/52598 [15:12<00:54, 54.43it/s]





 94%|█████████▍| 49675/52598 [15:12<00:53, 54.44it/s]





 94%|█████████▍| 49692/52598 [15:12<00:53, 54.45it/s]





 95%|█████████▍| 49712/52598 [15:12<00:52, 54.47it/s]





 95%|█████████▍| 49732/52598 [15:12<00:52, 54.49it/s]





 95%|█████████▍| 49750/52598 [15:12<00:52, 54.50it/s]





 95%|█████████▍| 49771/52598 [15:12<00:51, 54.52it/s]





 95%|█████████▍| 49790/52598 [15:13<00:51, 54.52it/s]





 95%|█████████▍| 49810/52598 [15:13<00:51, 54.53it/s]





 95%|█████████▍| 49826/52598 [15:13<00:50, 54.52it/s]





 95%|█████████▍| 49839/52598 [15:14<00:50, 54.52it/s]





 95%|█████████▍| 49849/52598 [15:14<00:50, 54.52it/s]





 95%|█████████▍| 49864/52598 [15:14<00:50, 54.53it/s]





 95%|█████████▍| 49879/52598 [15:14<00:49, 54.54it/s]




 98%|█████████▊| 51423/52598 [15:27<00:21, 55.45it/s]





 98%|█████████▊| 51439/52598 [15:27<00:20, 55.46it/s]





 98%|█████████▊| 51454/52598 [15:27<00:20, 55.47it/s]





 98%|█████████▊| 51463/52598 [15:27<00:20, 55.47it/s]





 98%|█████████▊| 51472/52598 [15:27<00:20, 55.47it/s]





 98%|█████████▊| 51480/52598 [15:28<00:20, 55.47it/s]





 98%|█████████▊| 51490/52598 [15:28<00:19, 55.48it/s]





 98%|█████████▊| 51498/52598 [15:28<00:19, 55.48it/s]





 98%|█████████▊| 51506/52598 [15:28<00:19, 55.47it/s]





 98%|█████████▊| 51513/52598 [15:28<00:19, 55.47it/s]





 98%|█████████▊| 51522/52598 [15:28<00:19, 55.48it/s]





 98%|█████████▊| 51529/52598 [15:29<00:19, 55.46it/s]





 98%|█████████▊| 51538/52598 [15:29<00:19, 55.46it/s]





 98%|█████████▊| 51544/52598 [15:29<00:19, 55.45it/s]





 98%|█████████▊| 51554/52598 [15:29<00:18, 55.45it/s]





 98%|█████████▊| 51564/52598 [15:29<00:18, 55.46it/s]





 98%|█████████▊| 51573/52598 [15:29<00:18, 55.46it/s]




100%|█████████▉| 52436/52598 [15:44<00:02, 55.55it/s]





100%|█████████▉| 52444/52598 [15:44<00:02, 55.54it/s]





100%|█████████▉| 52452/52598 [15:44<00:02, 55.54it/s]





100%|█████████▉| 52460/52598 [15:44<00:02, 55.54it/s]





100%|█████████▉| 52467/52598 [15:44<00:02, 55.54it/s]





100%|█████████▉| 52480/52598 [15:44<00:02, 55.55it/s]





100%|█████████▉| 52491/52598 [15:44<00:01, 55.55it/s]





100%|█████████▉| 52504/52598 [15:44<00:01, 55.56it/s]





100%|█████████▉| 52514/52598 [15:45<00:01, 55.56it/s]





100%|█████████▉| 52524/52598 [15:45<00:01, 55.57it/s]





100%|█████████▉| 52547/52598 [15:45<00:00, 55.59it/s]





100%|█████████▉| 52560/52598 [15:45<00:00, 55.59it/s]





100%|█████████▉| 52579/52598 [15:45<00:00, 55.61it/s]





100%|█████████▉| 52594/52598 [15:45<00:00, 55.61it/s]





100%|██████████| 52598/52598 [15:45<00:00, 55.62it/s]

In [16]:
# print(code)
for key,value in code.items():
    print(key, str(len(code[key])))

JavaSyntaxError 157794
code_col 157794
code_line 157794
code 157794
astcode 157794
database_source 157794


In [18]:
df = pd.DataFrame.from_dict(code)

In [24]:
import sys
sys.setrecursionlimit(160000)
outp_file = 'githubrepos.csv'
with open(outp_file, 'w', encoding='utf8', errors='replace') as f:
    df.to_csv(f, index=False, encoding = 'utf-8')
    
    
#     df.to_csv("githubrepos.csv", index=False, encoding = 'utf-8')